In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import *
from models import *

In [7]:
root = '../data-processing'
batch_size = 32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
attention = 0
_, _, dataset_test = load_data(root)
model = GraphSR(attention).to(device)
criterion = nn.L1Loss().to(device)
model.load_state_dict(torch.load(f'model_state/graphSR_attention_{attention}'))
loss, pred_super, gt_super, gt_low = evaluation(model, criterion, device, batch_size, dataset_test)
print(f'Test Loss, Attention={attention}: {round(loss, 4)}')

Test Loss, Attention=0: 0.0337


In [8]:
geodata_low_res = np.load('../data-processing/geodata_low_resolution.npy', allow_pickle=True)
geodata_super_res = np.load('../data-processing/geodata_super_resolution.npy', allow_pickle=True)

In [9]:
index = np.random.randint(0,874,1)
gt_low = gt_low.cpu()
gt_super = gt_super.cpu()
pred_super = pred_super.cpu()
vis_low = pd.DataFrame(np.concatenate([gt_low[index][0],  geodata_low_res[:,1].reshape(-1,1)],axis=1),
                       columns=['taxi_count', 'polygon'])
vis_super = pd.DataFrame(np.concatenate([gt_super[index][0],  
                                         pred_super[index][0], 
                                         geodata_super_res[:,2].reshape(-1,1)],axis=1),
                         columns=['taxi_count_gt', 'taxi_count_pred', 'polygon'])

In [11]:
vis_low.to_csv(f'visualization/vis_low_attention_{attention}.csv', header=True)
vis_super.to_csv(f'visualization/vis_super_attention_{attention}.csv', header=True)